In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
url = 'https://www.ymc.ru/price-list/'
num_list = 7
param = {'g': num_list, 's': 'a7574a2d-7bca-11e5-bbbf-b8975a69d0b3'}
response = requests.get(url, param, timeout=2)
soup_ = BeautifulSoup(response.text)

res_search = soup_.find_all(class_="price__new-position") #  class_="container content", class_="price__new-content",

#list_query = [] # лист для конкретного запроса

#for surgery in res_search:
    #type_= surgery.find('div')  # получения типа операции
    #name_ = surgery.find('div').find_next('div').text
    #price= surgery.find('div').find_next('div')
    #list_query.append({'Название': type_})

res_search

[]

**ЗАДАНИЕ_1**
**Результат только первой страницы!**
Вам необходимо написать функцию, которая будет основана на поиске по сайту habr.com. Функция в качестве параметра должна принимать список запросов для поиска (например, ['python', 'анализ данных']) и на основе материалов, попавших в результаты поиска по каждому запросу, возвращать датафрейм вида:


<дата> - <заголовок> - <ссылка на материал>

In [ ]:
%%time
import pandas as pd
import time
import requests
from bs4 import BeautifulSoup

my_search = ['Московский драматический театр', 'Процентная ставка ЦБ', 'Pandas']

def search_queries(search_list):
    full_list_qury = []
    for query in search_list:
        url = 'https://habr.com/ru/search'
        search_parameters = {'q': query, 'target_type': 'posts', 'order': 'relevance'}
        queries_soup = BeautifulSoup(requests.get(url, search_parameters).text)
        time.sleep(0.2)

        result_of_research = queries_soup.find_all(class_='tm-articles-list__item')  # можно: result_of_research = queries_soup.find_all('div', 'tm-article-snippet')
        list_query = [] # лист для конкретного запроса

        for x in result_of_research:
            link = x.find('a', 'tm-title__link').get('href')   # можно: link = x.find('h2').find_next('a').get('href')
            data = x.find('a').find_next('time').get('title')  # можно: data = x.find('a').find_next('time').text
            heading = x.find('a', 'tm-title__link').find_next('span').text
            like = x.find('div','tm-votes-meter tm-data-icons__item').find_next('span').get('title').strip('Всего голосов')

            full_url = 'https://habr.com' + link

            list_query.append({'Дата': data, 'Заголовок': heading, 'Ссылка на материал': full_url, 'Лайки': like})


        full_list_qury += list_query


    info_df = pd.DataFrame(full_list_qury)
    return info_df


search_1 = search_queries(my_search)


CPU times: user 260 ms, sys: 4.69 ms, total: 265 ms
Wall time: 5.75 s


In [ ]:
search_1.info()
search_1.iloc[[0, 3, 19, 20, 22, 23, 39, 40, 55 ], :]

**ЗАДАНИЕ_2** Результат для всех страниц!

кроме списка ключевых слов для поиска необходимо объявить параметр с количеством страниц поисковой выдачи. Т.е. при передаче в функцию аргумента 4 необходимо получить материалы с первых 4 страниц результатов;
в датафрейме должны быть столбцы с полным текстом найденных материалов и количеством лайков:
<дата> - <заголовок> - <ссылка на материал> - <текст материала> - <количество лайков>

In [ ]:
%%time

my_search_list_1 = ['Московский драматический театр', 'Процентная ставка ЦБ']
my_search_list_2 = ['python', 'анализ данных']
number_page = 4

import pandas as pd
import time
import requests
from bs4 import BeautifulSoup


#  ФУНКЦИЯ ФОРМИРУЕТ СПИСОК ИЗ URL_ССЫЛОК ВСЕХ СТРАНИЦ ЗАПРОСА
def search_queries(search_list, page):
    fullall_list_query = []

    for query in search_list:
      url = 'https://habr.com/ru/search/page1'
      search_parameters = {'q': query, 'target_type': 'posts', 'order': 'relevance'}
      for i in range(1, page + 1):
          url = url[:-1] + str(i)
          queries_soup = BeautifulSoup(requests.get(url, search_parameters).text)
          time.sleep(0.2)

          result_of_research = queries_soup.find_all('article', 'tm-articles-list__item')  # можно: result_of_research = queries_soup.find_all('div', 'tm-article-snippet')

          list_query = []

          for x in result_of_research:
              link = x.find('a', 'tm-title__link').get('href')  # можно: link =x.find('h2').find_next('a').get('href')

              list_query.append(link)

          list_full = list(map(lambda url_: 'https://habr.com' + url_, list_query))

          fullall_list_query += list_full

    return fullall_list_query

list_url_search_1 = search_queries(my_search_list_1, number_page) # полностью весь лист с ссылками по всем из списка запросов + по всем страница!!!

pd_url_search_1 = pd.DataFrame({'ССЫЛКА': list_url_search_1}) # датафрейм по всем запросам + по всем страницам!!!

list_url_search_2 = search_queries(my_search_list_2, number_page)

pd_url_search_2 = pd.DataFrame({'ССЫЛКА': list_url_search_2})


CPU times: user 3.16 s, sys: 165 ms, total: 3.33 s
Wall time: 30 s


In [ ]:
pd_url_search_2
pd_url_search_1

,ССЫЛКА
0,https://habr.com/ru/news/506622/
1,https://habr.com/ru/companies/selectel/article...
2,https://habr.com/ru/companies/lanit/articles/3...
3,https://habr.com/ru/news/808215/
4,https://habr.com/ru/news/565318/
...,...
155,https://habr.com/ru/articles/807443/
156,https://habr.com/ru/news/471632/
157,https://habr.com/ru/articles/48510/
158,https://habr.com/ru/articles/293380/


In [ ]:
# ИНФОРМАЦИОННАЯ ФУНКЦИЯ  url_ссылок/ ОБРАБОТКА ОШИБОК / ИСКЛЮЧЕНИЯ
%%time

def info_links(links_list):
  row_list = []
  for link in links_list:
    link_soup = BeautifulSoup(requests.get(link).text)
    time.sleep(0.2)

    try:
      data = link_soup.find('span').find_next('time').get('title')
    except:
      data = None
    try:
      heading = link_soup.find('h1').find_next('span').text
    except:
      heading = None

    text_info_link = link_soup.find_all(xmlns='http://www.w3.org/1999/xhtml')  # в основном текст находится в тегах 'p', но на сайте не мало и в теге 'div'
    full_info_text = '\n'.join(map(lambda txt: txt.text ,text_info_link))

    try:
      like = link_soup.find(class_='tm-svg-img tm-votes-meter__icon tm-votes-meter__icon tm-votes-meter__icon_appearance-article').find('title').text.strip('Всего голосов')
    except:
      like = None

    # ПАДАЕТ НА 32 индексе т.к там нет совсем  тега title и других тегов с лайками ( https://habr.com/ru/companies/smartengines/articles/820587/ )
    #like = link_soup.find('svg').find_next('title').find_next('title') # ХАБР
    #like = link_soup.find('svg').find_next('title').text # MENU ????????

    row_list.append({'ДАТА': data, 'ЗАГОЛОВОК': heading, 'ТЕКСТ МАТЕРИАЛА': full_info_text, 'ЛАЙКИ': like})



  return pd.DataFrame(row_list) # формирую сразу информационный датафрейм

df_info_search_1 = info_links(list_url_search_1)
df_info_search_2 = info_links(list_url_search_2)


CPU times: user 43.9 s, sys: 856 ms, total: 44.7 s
Wall time: 6min 43s


In [ ]:
pd_full_search_1 = df_info_search_1.join(pd_url_search_1)

pd_full_search_1.info()
pd_full_search_1.head(8)
pd_full_search_1.iloc[[31, 32, 33, 43, 44, 73, 74], :]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160 entries, 0 to 159
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ДАТА             159 non-null    object
 1   ЗАГОЛОВОК        160 non-null    object
 2   ТЕКСТ МАТЕРИАЛА  160 non-null    object
 3   ЛАЙКИ            157 non-null    object
 4   ССЫЛКА           160 non-null    object
dtypes: object(5)
memory usage: 6.4+ KB


,ДАТА,ЗАГОЛОВОК,ТЕКСТ МАТЕРИАЛА,ЛАЙКИ,ССЫЛКА
31,"2020-07-15, 20:18","Владимир Китов: «Сначала думал, что не смогу в...",\n\r\nВладимир Китов уже рассказывал о работе ...,17: ↑14 и ↓3,https://habr.com/ru/companies/dataart/articles...
32,"2024-04-22, 18:11",1C Game Studios рассказала о некоторых деталях...,Глава 1C Game Studios Альберт Жильцов и другие...,7: ↑7 и ↓0,https://habr.com/ru/news/809547/
33,"2023-08-30, 14:10",Александров Павел Сергеевич: советский математ...,Александров Павел Сергеевич был советским мате...,24: ↑20 и ↓4,https://habr.com/ru/companies/itglobalcom/arti...
43,"2016-07-31, 00:17","Хабрахабр в гостях у Александра Лямина, QRATOR",\nПолная версия видео доступна в конце публика...,61: ↑57 и ↓4,https://habr.com/ru/articles/306776/
44,None,"© 2006–2024, Habr",,None,https://habr.com/ru/companies/google/articles/...
73,"2019-06-06, 16:51",Взгляд изнутри: аспирантура в EPFL. Часть 4.1:...,"Посещая любую страну, важно не путать туризм ...",23: ↑22 и ↓1,https://habr.com/ru/articles/454958/
74,"2019-08-21, 10:40",Как живут и работают в Краснодаре,"Среди людей, работающих удалённо в ИТ и не тол...",47: ↑42 и ↓5,https://habr.com/ru/companies/skyeng/articles/...


In [ ]:
pd_full_search_2.to_excel('pd_full_search_2.xlsx', index=False)

# ПРОПУСКИ И ИХ УДАЛЕНИЕ

In [ ]:
pd_full_search_2 = df_info_search_2.join(pd_url_search_2)

pd_full_search_2.info()
pd_full_search_2.loc[pd.isna(pd_full_search_2).any(axis=1)] # Показывает все строки с None!!!


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160 entries, 0 to 159
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ДАТА             159 non-null    object
 1   ЗАГОЛОВОК        156 non-null    object
 2   ТЕКСТ МАТЕРИАЛА  160 non-null    object
 3   ЛАЙКИ            149 non-null    object
 4   ССЫЛКА           160 non-null    object
dtypes: object(5)
memory usage: 6.4+ KB


,ДАТА,ЗАГОЛОВОК,ТЕКСТ МАТЕРИАЛА,ЛАЙКИ,ССЫЛКА
2,"2024-06-18, 10:16",Установка Python,Версии PythonНа данный момент существует множе...,None,https://habr.com/ru/articles/822557/
4,"2024-06-14, 15:32",Лучшие библиотеки Python для Data Science в 20...,Python — один из самых распространенных языков...,None,https://habr.com/ru/companies/productstar/arti...
6,"2024-06-16, 10:15",Дроиды и питоны: работаем с ADB на Python,"Однажды, в одну из бессонных ночей, я думал, к...",None,https://habr.com/ru/articles/822099/
8,"2024-06-18, 19:46",Основы архитектуры для джунов: построение масш...,Когда речь идет о создании масштабируемых и по...,None,https://habr.com/ru/articles/816355/
11,"2024-06-19, 10:12",Симметричная индексация в Python,"Иногда, изучая Python, можно наткнуться на вещ...",None,https://habr.com/ru/articles/818479/
13,"2024-06-11, 11:18",Динамическое программирование на Python,"Привет, Хабр!Динамическое программирование пол...",None,https://habr.com/ru/companies/otus/articles/81...
16,"2023-05-19, 15:04",None,,2: ↑2 и ↓0,https://habr.com/ru/companies/evrone/news/748986/
20,"2024-06-19, 10:51",Использование Annotated в Python,Всем привет. Ранее мы с вами разбирали универс...,None,https://habr.com/ru/companies/tensor/articles/...
21,"2024-06-19, 12:12",Как работать с объектным хранилищем на Python,\r\nФайлы в проекте можно хранить разными спос...,None,https://habr.com/ru/companies/selectel/article...
26,"2022-02-18, 12:01",None,,2: ↑2 и ↓0,https://habr.com/ru/companies/epam_systems/new...


In [ ]:
pd_full_search_2.isna().mean()*100  # считает колличество пропусков / isnull тот же метод!!!

ДАТА               0.625
ЗАГОЛОВОК          2.500
ТЕКСТ МАТЕРИАЛА    0.000
ЛАЙКИ              6.875
ССЫЛКА             0.000
dtype: float64

In [ ]:
pd_full_search_2.dropna() # удаление пропусков во свсём датафрейме

In [ ]:
pd_full_search_2.dropna(thresh=3) # удаление пропусков кроме строк которые содержит данные в 3 столбцах

In [ ]:
pd_full_search_2.drop(['ТЕКСТ МАТЕРИАЛА'], axis=1) # удаление столбца

In [ ]:
pd_full_search_2['ЛАЙКИ']=pd_full_search_2['ЛАЙКИ'].fillna('нет данных')
pd_full_search_2

,ДАТА,ЗАГОЛОВОК,ТЕКСТ МАТЕРИАЛА,ЛАЙКИ,ССЫЛКА
0,"2022-06-18, 13:48","Куда пойти, когда только выучил Python: советы...","Python любят начинающие, потому что его относи...",7: ↑2 и ↓5,https://habr.com/ru/companies/netologyru/artic...
1,"2024-04-15, 10:00",Какой язык программирования выбрать? Обзор Python,Шестая статья нашего цикла про языки программи...,19: ↑10 и ↓9,https://habr.com/ru/companies/kryptonite/artic...
2,"2024-06-18, 10:16",Установка Python,Версии PythonНа данный момент существует множе...,нет данных,https://habr.com/ru/articles/822557/
3,"2024-05-21, 14:45",Создаём безукоризненную автодокументацию кода ...,У дата-сайентиста множество обязанностей в каж...,12: ↑12 и ↓0,https://habr.com/ru/companies/netologyru/artic...
4,"2024-06-14, 15:32",Лучшие библиотеки Python для Data Science в 20...,Python — один из самых распространенных языков...,нет данных,https://habr.com/ru/companies/productstar/arti...
...,...,...,...,...,...
155,"2014-07-07, 16:24",Обзор наиболее интересных материалов по анализ...,\r\nПрошлый выпуск обзора был полностью посвящ...,24: ↑23 и ↓1,https://habr.com/ru/articles/228989/
156,"2015-03-01, 19:07",Обзор наиболее интересных материалов по анализ...,\r\nПредставляю вашему вниманию очередной выпу...,20: ↑19 и ↓1,https://habr.com/ru/articles/251829/
157,"2014-12-07, 16:38",Обзор наиболее интересных материалов по анализ...,\r\nПредставляю вашему вниманию очередной выпу...,34: ↑32 и ↓2,https://habr.com/ru/articles/245195/
158,"2015-01-04, 19:50",Обзор наиболее интересных материалов по анализ...,\r\nПредставляю вашему вниманию очередной выпу...,22: ↑20 и ↓2,https://habr.com/ru/articles/247321/


Здравствуйте.

1.   **Задание номер один.** По первой функции будет отрабатывать без ошибок. независимо что запрашиваем. Можно найти по первой странице все что нужно и без ошибок.
2.   **Задание номер два:**

ФУНКЦИЯ КОТОРАЯ СОБИРАЕТ ВСЕ ССЫЛКИ ПО СТРАНИЦАМ ОТРАБАТЫВАЕТ ПРАВИЛЬНО (ПРОВЕРЯЛ ЭТИ ССЫЛКИ ПО ПОИСКУ, ПОСТРАНИЧНО КАЖДУЮ И ПО КАЖДОМУ ЗАПРОСУ КОТОРЫЕ В ЛИСТЕ ЗАПРОСОВ)

По второй функции - Все намного для меня сложнее.

Лайки я так и не смог получить - поставил habr. Хотя нужные можно найти по такому классу: like = link_soup.find(class_='tm-svg-img tm-votes-meter__icon tm-votes-meter__icon tm-votes-meter__icon_appearance-article').find('title').text

И по запросу 'Московский драматический театр я их получил частично!

Интересно то что все находит, но на 33 ссылке (индекс 32)
https://habr.com/ru/companies/smartengines/articles/820587/- возникает ошибка. И понятно - там совсем другой класс и/ или тег. Если меняю на него чтоб адекватно находил лайк данной ссылки, то уже будут ошибки по 1-32 ссылкам лайки  не отработают.

class="tm-article-rating tm-data-icons__item - общий класс - для всех ссылок - но безусловно я получу лайки NONE для всех ссылок, но уже без ошибок!!!

Позапросу 'Московский драматический театр'
если выбираю три страницы, то код падает по ссылки 43(42 индекс) (	https://habr.com/ru/companies/google/articles/131270/) - страницы уже не существует:((((( ОСТАЛЬНЫЕ СТРАНИЦЫ НАХОДИТ.

Дальше, если поиск по PYTHON то ошибка падает по дате т.к 66 ссылки (67 индекс уже НЕ СУЩЕСТВУЕТ!!!) (3 страница поиска)
https://habr.com/ru/companies/yandex/articles/148443/
Я НЕДЕЛЮ РЕШАЛ ЭТУ ЗАДАЧУ. ОКАЗЫВАЮТСЯ ВОТ ОСНОВЫНЕ ПРИЧИНЫ ПАДЕНИЯ КОДА.

ТАКИМ ОБРАЗОМ РАБОТА ВТОРОЙ ФУНКЦИИ ЗАВИСИТ ОТ КОЛИЧЕСВА СТРАНИЦ, САМОГО ЗАПРОСА - ЧТО ИЩЕМ + если искомые данные находятся по одинаковым тегам и классам. ЧЕМ БОЛЬШЕ СТРАНИЦ, ТЕМ ВЫШЕ ВЕРОЯТНОСТЬ ПАДЕНИЯ КОДА...УНИВЕРСАЛЬНОЙ НЕТ ИЛИ НУЖНО ПРАВИЛЬНО НАУЧИТСЯ ОБРАБАТЫВАТЬ ИШИБКИ КОДА.

СКАЖИТЕ, КАК ТОГДА ОБРАБАТЫВАТЬ ПОДОБНЫЕ  ОШИБКИ ЧТОБ КОД НЕ ПАДАЛ. ВМЕСТО НЕНАЙДЕННОГО ЛАЙКА или отсутсвия страницы  ПРОПИСАТЬ как none(или что-то другое)


TRY - EXCEPT - КАК ЕГО ПРОПИСАТЬ ПРАВИЛЬНО?


